In [ ]:
# Clone realESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Clone BSRGAN
!git clone https://github.com/cszn/BSRGAN.git

!rm -r SwinIR
# Clone SwinIR
!git clone https://github.com/JingyunLiang/SwinIR.git
!pip install timm

# Download the pre-trained models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P experiments/pretrained_models

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 755, done.
remote: Total 755 (delta 0), reused 0 (delta 0), pack-reused 755
Receiving objects: 100% (755/755), 5.37 MiB | 17.90 MiB/s, done.
Resolving deltas: 100% (410/410), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 24.6 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214816 sha256=4d23337196beb33827f67626bff7d36ae072f7824b9a42ace207cfd964c53a2d
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.9 MB/s eta 0:00:00
     ━━━━

In [ ]:
import pandas as pd
from google.colab import files
ff = files.upload()
for file_name in ff.keys():
    df1 = pd.read_excel(file_name, 'EN Coloring',usecols="A")
print(df1)

Saving KW Research Pearlpaint.xlsx to KW Research Pearlpaint (3).xlsx
                               Keyword
0                       coloring pages
1             christmas coloring pages
2              coloring pages for kids
3             coloring pages printable
4             halloween coloring pages
...                                ...
9991               jason coloring page
9992             jessie coloring pages
9993          jesus coloring pages lds
9994    jesus feeds 5000 coloring page
9995  jesus resurrection coloring page

[9996 rows x 1 columns]


In [ ]:
import requests
import os

def download_images(keywords, num_images, download_path,name):
    # Initializing the search URL
    search_url = f"https://www.google.com/search?q={'+'.join(keywords)}&source=lnms&tbm=isch"

    # Sending HTTP GET request
    response = requests.get(search_url)

    # Parsing the response HTML using BeautifulSoup
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Extracting image URLs using the 'img' tag
    img_tags = soup.find_all("img")
    urls = [img["src"] for img in img_tags]

    # Downloading images
    counter = 0
    for url in urls:
        if counter == num_images:
            break
        try:
            # Sending HTTP GET request to download the image
            response = requests.get(url)
            response.raise_for_status()

            # Saving the image to a file
            with open(os.path.join(download_path, f"image{name}.jpg"), "wb") as f:
                f.write(response.content)

            counter += 1
        except Exception as e:
            print(f"Error downloading image {counter}: {str(e)}")

# Example usage
for x in df1['Keyword'].iloc[0:100]:
  x=str(x)
  keywords = [x]  # Keywords to search for
  num_images = 1           # Number of images to download
  download_path = "/content/images"  # Path to download the images

  # Creating the download directory if not exists
  os.makedirs(download_path, exist_ok=True)

  # Downloading images
  download_images(keywords, num_images, download_path,x)


In [ ]:
import os
import glob
from google.colab import files
import shutil
print(' Note1: You can find an image on the web or download images from the RealSRSet (proposed in BSRGAN, ICCV2021) at https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/RealSRSet+5images.zip.\n Note2: You may need Chrome to enable file uploading!\n Note3: If out-of-memory, set test_patch_wise = True.\n')

# test SwinIR by partioning the image into patches
test_patch_wise = False

# to be compatible with BSRGAN
!rm -r BSRGAN/testsets/RealSRSet
upload_folder = 'BSRGAN/testsets/RealSRSet'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
#uploaded = files.upload()
uploaded = "/content/images/"
picture_list =  os.listdir(uploaded)
#for x in picture_list:
#  dst_path = os.path.join(upload_folder, x)
#  print(f'Move {uploaded} to {dst_path}')
  #shutil.move(uploaded, dst_path)
  # SwinIR
  #if test_patch_wise:
  #  !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale 4 --tile 800
  #else:
!python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq /content/images --scale 4
  #shutil.move('results/swinir_real_sr_x4', 'results/SwinIR')


 Note1: You can find an image on the web or download images from the RealSRSet (proposed in BSRGAN, ICCV2021) at https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/RealSRSet+5images.zip.
 Note2: You may need Chrome to enable file uploading!
 Note3: If out-of-memory, set test_patch_wise = True.

loading model from experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Testing 0 imageadult coloring page
Testing 1 imageadult coloring pages printable
Testing 2 imageadult coloring pages
Testing 3 imageamong us coloring pages
Testing 4 imageanimal coloring pages
Testing 5 imageanime coloring pages
Testing 6 imagebaby shark coloring pages
Testing 7 imag

In [ ]:
# Download the results
zip_filename = 'SwinIR_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ls

 assets/                            options/
 BSRGAN/                            README_CN.md
 CODE_OF_CONDUCT.md                 README.md
 cog_predict.py                     realesrgan/
 cog.yaml                           realesrgan.egg-info/
 docs/                              requirements.txt
 experiments/                       results/
 inference_realesrgan.py            scripts/
 inference_realesrgan_video.py      setup.cfg
 inputs/                            setup.py
'KW Research Pearlpaint (1).xlsx'   SwinIR/
'KW Research Pearlpaint (2).xlsx'   SwinIR_result.zip
'KW Research Pearlpaint (3).xlsx'   tests/
'KW Research Pearlpaint.xlsx'       VERSION
 LICENSE                            weights/
 MANIFEST.in
